In [15]:
cafe = bytes('cafe', encoding='utf-8')
print(cafe)

for i,b in enumerate(cafe) : 
    print(f"cafe[{i}] : {b}")
    
cafe_arr = bytearray('cafe', encoding='utf-8')
print("\n",cafe_arr, sep = '')

for i,b in enumerate(cafe_arr) : 
    print(f"cafe[{i}] : {b}")
    
    
# bytearray[:1]은 bytearray를 반환한다.
print("\n", cafe_arr[:1], sep = '')

# s[0] == s[:1] 이 참이 되는 경우는 str이 유일하다.


b'cafe'
cafe[0] : 99
cafe[1] : 97
cafe[2] : 102
cafe[3] : 101

bytearray(b'cafe')
cafe[0] : 99
cafe[1] : 97
cafe[2] : 102
cafe[3] : 101

bytearray(b'c')


In [17]:
# bytes는 형변환을 지원한다.

import array
nums = array.array('h', [-2,-1,0,1,2])
octets = bytes(nums)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

# Codec 

In [27]:
# 어떤 codec을 사용하냐에 따라 인코딩이 달라진다.

for codec in ['utf_8', 'utf_16', 'cp949'] :
    print(codec, "사과".encode(codec), sep = '\t')

utf_8	b'\xec\x82\xac\xea\xb3\xbc'
utf_16	b'\xff\xfe\xac\xc0\xfc\xac'
cp949	b'\xbb\xe7\xb0\xfa'


#### 파일을 작성할 때 작성한 codec과 읽을 때 사용한 codec이 다르면
#### 에러를 발생하거나 문자가 깨질 수 있다.
#### 따라서 codec을 미리 통일할 필요가 있다. (특히 여러 사람과 작업하게 된다면)

In [37]:
with open("text.txt", 'w', encoding = 'cp949') as f :
    f.write("안녕하세요.")

try :
    f = open("text.txt", 'r', encoding = 'utf-8').read()
except Exception as e : 
    print(e)
    f = open("text.txt", 'r', encoding = 'cp949').read()
    
print(f)

'utf-8' codec can't decode byte 0xbe in position 0: invalid start byte
안녕하세요.


#### sys와 locale 에서 default codec을 확인할 수 있다.

#### locale은 현재 코드 작성 환경의 codec을 보여준다. 
#### sys 은 표준입출력의 codec을 보여준다.
#### sys.~.isatty() 은 입출력이 콘솔(터미널) 에서 작동하는지 여부를 알려준다.

#### 출력을 보면 현재 필자의 코드 작성 환경이 주피터 노트북으로 리다이렉션 됬으므로 isatty() 가 False로 나온다.
####   리다이렉션 : 표준입출력을 사용자 지정 위치로 우회하는 것을 말한다.
#### locale 이 cp949 이므로 my_file도 cp949로 encoding 되었음을 알 수 있다.
#### 하지만 sys 상에서는 utf-8을 디폴트로 사용하고 있다.
#### 따라서 my_file을 주피터 노트북 환경이 아닌 터미널에서 실행하여 파일을 읽어들일 경우 오류가 발생할 것을 예상할 수 있다.
#### 이 경우에는 encoding = 'cp949' 을 설정해야 오류를 피할 수 있다.

#### 결론 : 기본 인코딩에 의존하지 말고 인코딩을 명시하자.

In [68]:
import sys, locale

expressions = """ 
        locale.getprefererdencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split() :
    value = eval(expression)
    print(f"{expression:>30} -> {str(value)}")

 locale.getpreferredencoding() -> cp949
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> cp949
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> UTF-8
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> utf-8
           sys.stderr.isatty() -> False
      sys.getdefaultencoding() -> utf-8
   sys.getfilesystemencoding() -> utf-8


In [5]:
s1 = 'café'
s2 = "cafe\u0301"
print(s1,s2)
print(len(s1), len(s2))
print(s1 == s2)

café café
5 5
True


# 비유니코드 문자열 정렬

In [39]:
# 포르투갈어 등 라틴어에서는 갈고리나 악센트가 문자에 붙어 정렬이 비정상적일 수 있다.
import numpy as np

fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
order = [3,4,1,2,0]
print('sorted() 사용 : ', sorted(fruits))
print("올바른 정렬 : ", np.array(fruits)[order])


sorted() 사용 :  ['acerola', 'atemoia', 'açaí', 'cajá', 'caju']
올바른 정렬 :  ['açaí' 'acerola' 'atemoia' 'cajá' 'caju']


#### 특수문자가 포함된 문자열을 locale.strxfrm() 함수를 통해 현지어로 바꿀 수있다.
#### 그러기 위해선 locale.setlocale(locale.LC_COLLATE, <지역_언어>) 을 호출해야한다.
#### 하지만 locale.setlocale은 시스템 전역에 영향을 미치므로 이는 추천하지 않는 방법이다.
#### 또한 OS에서 이 설정을 지원해야 한다. 그렇지 않으면 오류가 발생하거나 올바르게 정렬되지 않을 수 있다.

In [41]:
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

# 윈도우 환경에서는 locale 설정이 지원되고 올바르게 정렬된 것을 확인할 수 있다.
print(sorted(fruits, key = locale.strxfrm))

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']


#### 이 문제를 겪고 천재 제임스 토버님께서 라이브러리를 만드셨다.
#### 사용법은 간단하다. 그냥 key 에 'sort_key' method 를 주면 된다.   
#### 정렬 방식을 커스트마이징 하고 싶다면 Collator()에 직접 만든 대조테이블을 제공하면 된다.
#### 기본적으로 라이브러리 내의 [allkeys.txt](https://github.com/jtauber/pyuca)를 사용한다.
#### 이는 유니코드 6.3.0 에서 제공하는 [기본 유니코드 대조 요소 테이블](http://bit.ly/1IqAk54)의 사본이다

In [45]:

import pyuca 
coll = pyuca.Collator()
print(sorted(fruits, key = coll.sort_key))

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']


# 유니코드 데이터 베이스

#### 유니코드 데이터베이스에는 코드 포인트를 문자명으로 매핑하는 테이블 뿐만 아니라 메타데이터도 담고 있다.
#### unicodedata.numeric()이 재밌는데 문자가 나타내는 숫자를 저장해놓았다.

In [83]:
import unicodedata
import re

# 모든 유니코드 십진수의 정규표현식
re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample :
    decode = hex(ord(char))[2:]
    print(f"U+{decode:0>4}",
          f"{char:^6}",
          're_dig' if re_digit.match(char) else '-',
          'isdig' if char.isdigit() else '-',
          'isnum' if char.isnumeric() else '-',
          f"{unicodedata.numeric(char):5.2f}",
          unicodedata.name(char),
          sep = '\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


## 정규표현식
### bytes : \d, \w 와 같은 패턴은 아스키 문자만 매칭된다.
### str   : 아스키 문자 이외에 유니코드 숫자나 문자도 매칭된다.

In [11]:
import re

re_numbers_str = re.compile(r"\d+")
re_words_str = re.compile(r"\w+")
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = "Ramanujan saw \u0be7\u0be7\u0bed\u0be8\u0bef as 1729 = 1³ + 12³ = 9³ + 10³."

text_bytes = text_str.encode('utf-8')

print("Text", text_str, sep = '\n   ')
print("Numbers")
print("   str :", re_numbers_str.findall(text_str))
print("   bytes :", re_numbers_bytes.findall(text_bytes))
print("Words")
print("   str :", re_words_str.findall(text_str))
print("   bytes :", re_words_bytes.findall(text_bytes))

Text
   Ramanujan saw ௧௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.
Numbers
   str : ['௧௧௭௨௯', '1729', '1', '12', '9', '10']
   bytes : [b'1729', b'1', b'12', b'9', b'10']
Words
   str : ['Ramanujan', 'saw', '௧௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
   bytes : [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']
